In [34]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from keras.layers import Input, Dense, Lambda, Flatten, Reshape
from keras.layers import Conv1D
from keras.models import Model

from keras import backend as K

# n_features = 4
# n_runs = 100
# seq_length = [180, 360, 540, 720, 900]
# labels = pd.read_csv("Data/Boat_nominal_data/Boat_mix_len_labels.csv")
# labels = labels.drop(columns="Unnamed: 0") 

# labels = np.array(labels)
# 

# def read_sequences():
#     run_list_mix = []
#     run_list_ordered = []
#     labels_reordered = []
#     for index in range(n_runs):
#         run_csv = pd.read_csv("Mix_sequences_var_length/run^"+str(index))
#         run_csv = run_csv.drop(columns=['Unnamed: 0'])
#         run_list_mix.append(run_csv)
#     for index, length in enumerate(seq_length):
#         run_list_ordered.append([])
#         for run_index, run in enumerate(run_list_mix): 
#             if len(run) == length:
#                 run_list_ordered[index].append(run)    
#                 labels_reordered.append(labels[run_index])
#     return run_list_ordered, labels_reordered
#     
#     
#     
# def stadard_sequences(sequences):
#     for index, seqs in enumerate(sequences):
#         for i, seq in enumerate(seqs):
#             seqs[i] = MinMaxScaler(feature_range=[0, 1]).fit_transform(seq)
#     return sequences                    
# 
# 
# trainX_sequences, labels_ordered = read_sequences()
# trainX_sequences = stadard_sequences(trainX_sequences)
# for i, leng in enumerate(seq_length):
#     trainX_sequences[i] = np.reshape(trainX_sequences[i], (-1, leng, n_features))

# def train_generator():
#     index = 0
#     while True:
#         x_train = trainX_sequences[index]
#         index += 1
#         yield (x_train, x_train) 


In [35]:
n_features = 4 

n_runs = 1000

labels = pd.read_csv("Data/Boat_nominal_data/Boat_mix_len_labels.csv")
labels = labels.drop(columns="Unnamed: 0") 
labels = np.array(labels)
max_len = 0
def get_max_len(sequence_list):
    m_len = 0
    for seq in sequence_list:
        if len(seq) > m_len:
            m_len = len(seq)
    max_len = m_len
    print(max_len)
    return m_len


def construct_matrix(sequence_list):
    max_len = get_max_len(sequence_list)
    train_matrix = np.zeros(shape=(n_runs, max_len, n_features))
    for index,run in enumerate(sequence_list):
        train_matrix[index, :len(run)] = run
    print(train_matrix.shape)
    return train_matrix
    
    
def stadard_sequences(sequences):
    for i, seq in enumerate(sequences):
        sequences[i] = MinMaxScaler(feature_range=[0, 1]).fit_transform(seq)
        #sequences[i] = StandardScaler().fit_transform(seq)
    return sequences       

def read_sequences():
    run_list_mix = []
    for index in range(n_runs):
        run_csv = pd.read_csv("Mix_sequences_var_length/run^"+str(index))
        run_csv = run_csv.drop(columns=['Unnamed: 0'])
        run_list_mix.append(run_csv)
    stand_sequences = stadard_sequences(run_list_mix)
    padded_matrix = construct_matrix(stand_sequences)
    return padded_matrix


train_matrix = read_sequences() 


900
(1000, 900, 4)


In [50]:

from keras.layers import Input, LSTM, RepeatVector, Conv2DTranspose
from keras.losses import mse
from keras.models import Model


filters = 50
intermediate_dimension = 30 
latent_dim = 10

def Conv1DTranspose(input_tensor, filters, kernel_size,last, strides=2, padding='same'):
        if last:
            activation = 'linear'
        else:
            activation = 'relu'
        x = Lambda(lambda x: K.expand_dims(x, axis=2))(input_tensor)
        x = Conv2DTranspose(filters=filters, kernel_size=(kernel_size, 1), 
                            strides=(strides, 1), padding=padding,
                            activation=activation)(x)
        x = Lambda(lambda x: K.squeeze(x, axis=2))(x)
        return x


def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon


def repeat(x):
    steps_matrix = K.ones_like(x[0][:, :, :1])
    latent_matrix = K.expand_dims(x[1], axis=1)
    return K.batch_dot(steps_matrix, latent_matrix)


def create_vae():
    print(max_len)
    inputs = Input(shape=(900, n_features))
    x = inputs
    
    for i in range(2):
        x = Conv1D(filters=filters,
                   kernel_size=2,
                   strides=2,
                   padding='same')(x)
    
    shape = K.int_shape(x)
    
    #before_flattening = LSTM(intermediate_dimension, return_sequences=True)(x)
    #x_flat = LSTM(intermediate_dimension)(before_flattening)
    x = Flatten()(x)
    
    
    embeddings = Dense(latent_dim)(x)
    
    z_mean = Dense(latent_dim, name='z_mean',)(embeddings)
    z_log_var = Dense(latent_dim, name='z_log_var')(embeddings)
    z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])
    
    encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
    
    latent_inputs = Input(shape=(latent_dim,), name='latent_inputs')
    #x = Lambda(repeat)([before_flattening, z])
    x = Dense(shape[1]*shape[2])(latent_inputs)
    x = Reshape((shape[1],shape[2]))(x)
    
    
    print(K.int_shape(x))
    for i in range(2):
        x = Conv1DTranspose(input_tensor=x,
                            filters=filters,
                            kernel_size=2,
                            last=False,
                            padding='same')
    
    
    outputs = Conv1DTranspose(input_tensor=x,
                              filters=n_features,
                              kernel_size=2,
                              strides=1,
                              last=True,
                              padding='same')
    
    encoder.summary()
    decoder = Model(latent_inputs, outputs)
    decoder.summary()
    outputs = decoder(encoder.outputs[2])
    reconstruction_loss = mse(K.flatten(inputs), K.flatten(outputs))
    #reconstruction_loss *= sequence_length*4
    kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), 
                             axis=-1)
    loss = K.mean(reconstruction_loss+0*kl_loss)
    vae = Model(inputs, outputs, name='vae')
    vae.add_loss(loss)
    #vae.summary()
    vae.compile(optimizer='adam')
    return vae, encoder


def create_ae():
    inputs = Input(shape=(900, n_features))
    x = inputs
    for i in range(2):
        x = Conv1D(filters=filters, kernel_size=2, strides=2,
                   activation='relu', padding='same')(x)
    shape = K.int_shape(x)   
    before_flattening = x #LSTM(filters, return_sequences=True)(x)   
    #encoded = LSTM(intermediate_dimension)(before_flattening)
    encoded = Flatten()(x)
    encoded = Dense(latent_dim)(encoded)
    #x = Lambda(repeat)([before_flattening, encoded])
    
    latent_inputs = Input(shape=(latent_dim,), name='latent_inputs')
    x = Dense(shape[1]*shape[2])(latent_inputs)
    x = Reshape((shape[1],shape[2]))(x)
    
    for i in range(2):
        x = Conv1DTranspose(input_tensor=x, filters=filters,
                            kernel_size=2, padding='same', last=False)
       
       
    #decoded = LSTM(n_features, return_sequences=True)(x)
    
    output = Dense(n_features)(x)
    
    encoder = Model(inputs, encoded)
    decoder = Model(latent_inputs, output)
    
    output = decoder(encoder.output)
    sequence_autoencoder = Model(inputs, output)
    sequence_autoencoder.summary()
    sequence_autoencoder.compile(optimizer='adam', loss='mse')
    return sequence_autoencoder, encoder


model, encoder = create_vae()


0
(None, 225, 50)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           (None, 900, 4)       0                                            
__________________________________________________________________________________________________
conv1d_31 (Conv1D)              (None, 450, 50)      450         input_15[0][0]                   
__________________________________________________________________________________________________
conv1d_32 (Conv1D)              (None, 225, 50)      5050        conv1d_31[0][0]                  
__________________________________________________________________________________________________
flatten_12 (Flatten)            (None, 11250)        0           conv1d_32[0][0]                  
__________________________________________________________________________________________________
dense_32 (

In [52]:
from keras.callbacks import ModelCheckpoint


def train():
    print(train_matrix.shape)
    model.fit(train_matrix, epochs=100, verbose=1)
    model.save_weights("Models/Weights/AE_CONV_BATCH_1000_diff_len_980_MATRIX_LEN.hdf5")


train()


(1000, 900, 4)


Epoch 1/100


  32/1000 [..............................] - ETA: 1:46 - loss: 0.2031

  64/1000 [>.............................] - ETA: 54s - loss: 0.1937 

  96/1000 [=>............................] - ETA: 36s - loss: 0.2019

...

 896/1000 [=========================>....] - ETA: 0s - loss: 1.6500e-04

 928/1000 [==========================>...] - ETA: 0s - loss: 1.6517e-04

 960/1000 [===========================>..] - ETA: 0s - loss: 1.6484e-04

 992/1000 [============================>.] - ETA: 0s - loss: 1.6477e-04

1000/1000 [==============================] - 6s 6ms/step - loss: 1.6580e-04


In [21]:

model.load_weights("Models/Weights/AE_CONV_BATCH_cycles_MATRIX_Len.hdf5")


In [135]:
# from sklearn.decomposition import PCA
# 
# encodings = []
# for i,seqs in enumerate(trainX_sequences): 
#     if len(seqs) != 0:
#         encodings.append(encoder.predict(seqs))
#     
# def squeeze_vectors(vec):
#     enc_sq = []
#     for i in vec:
#         for j in i:
#             enc_sq.append(j)
#             vec_squeezed = np.array(enc_sq)
#     return vec_squeezed
#     
# def reshape_vectors():
#     means, vars, z = [], [], []
#     for encoded_seq in encodings:
#         means.append(encoded_seq[0])
#         vars.append(encoded_seq[1])
#         z.append(encoded_seq[2])
#     means = squeeze_vectors(means)
#     vars = squeeze_vectors(vars)
#     z = squeeze_vectors(z)
#     return means, vars, z
# 
# 
# means_encs, log_var_encs, z_encs = reshape_vectors()
# 
# print(means_encs.shape,log_var_encs.shape, z_encs.shape )
# 
# def return_mask(num, labs):
#     arg = np.squeeze(np.argwhere(labs == num))
#     return arg
# 
#     
# masks = [return_mask(num, np.array(labels_ordered))[:, 0] for num in range(0, 9)]


In [54]:

from sklearn.decomposition import PCA

encodings = encoder.predict(train_matrix)
enc_mean, enc_var, z_enc = encodings[0], encodings[1], encodings[2]
print(enc_mean.shape, enc_var.shape, z_enc.shape)

(1000, 10) (1000, 10) (1000, 10)


In [55]:


def return_mask(num, labs):
    arg = np.squeeze(np.argwhere(labs == num))
    return arg


masks = [return_mask(num, np.array(labels))[:, 0] for num in range(0, 9)]


In [57]:

from mpl_toolkits.mplot3d import Axes3D  


def plot_pca(title, i): 
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    markers = ['o', 'o', 'o', 'o', '^', '^', '^', '^', '^', '^']
    for index, mask in enumerate(masks):
        ax.scatter(principalComponents[:, 0][mask], 
                   principalComponents[:, 1][mask],  
                   principalComponents[:, 2][mask], marker=markers[index])
        #break
    plt.legend(labels=np.arange(0, 9))
    plt.title(str(title))
    plt.show()
    
    for mask in masks:
        plt.scatter(x=principalComponents[:, 0][mask], 
                    y=principalComponents[:, 1][mask],
                    alpha=0.5)
        #break
    plt.legend(labels=np.arange(0, 9))
    plt.title(str(title))
    plt.show()


enc_list = [enc_mean, enc_var, z_enc]
titles = ["MEAN","LOG_VAR","SAMPLED"]
for i,enc in enumerate(enc_list):
    scaler = StandardScaler()
    enc_input = scaler.fit_transform(enc) 
    pca = PCA(3)
    principalComponents = pca.fit_transform(enc_input)
    print(principalComponents.shape)
    print(pca.explained_variance_ratio_)
    plot_pca('Sequences'+titles[i], 0)
    
    principalComponents = enc
    plot_pca('Sequences_Not_Pca'+titles[i], 0)
    


(1000, 3)
[0.6383846  0.18312241 0.11776037]


(1000, 3)
[0.5209906  0.3191389  0.11770153]


(1000, 3)
[0.63815576 0.18310663 0.11774333]


In [ ]:
# 
# inputs = Input(shape=(None, n_features), name="Input")
# x = inputs
# for i in range(2):
#     x = Conv1D(filters=filters, kernel_size=2, strides=2,
#                activation='relu', padding='same')(x)
#    
# before_flattening = LSTM(filters, return_sequences=True)(x)   
# x_flat = LSTM(intermediate_dimension)(before_flattening)
# 
# z_mean = Dense(latent_dim, name='z_mean')(x_flat)
# z_log_var = Dense(latent_dim, name='z_log_var')(x_flat)
# z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])
# 
# encoder = Model(inputs, [z_mean, z_log_var, z])
# 
# 
# 
# x = Lambda(repeat)([before_flattening, z])
# 
# for i in range(2):
#     x = Conv1DTranspose(input_tensor=x, filters=filters,
#                         kernel_size=2, padding='same', last=False)
#    
# #decoded = LSTM(n_features, return_sequences=True)(x)
# 
# outputs = Dense(n_features)(x)
# 
# 
# 
# vae = Model(inputs, outputs)
# 
# 
# reconstruction_loss = mse(K.flatten(inputs), K.flatten(outputs))
# kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
# vae_loss = K.mean(reconstruction_loss + 0*kl_loss)
# vae.add_loss(vae_loss)
# 
# vae.summary()
# vae.compile(optimizer='adam')


In [58]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, random_state=0)

tsne_obj= tsne.fit_transform(enc_input)
print(tsne_obj.shape)

for mask in masks:
    plt.scatter(x=tsne_obj[:, 0][mask], 
                y=tsne_obj[:, 1][mask],
                alpha=0.5)
plt.show()

(1000, 2)
